In [ ]:
import numpy as np
from pywarpx import picmi

from utils import cosd, sind
from space_analysis.simulation.warpx import HybridSimulation

In [ ]:
constants = picmi.constants

Support `gaussian_parse_momentum_function`?


In [ ]:
def init_field_with_plasma(
    k,
    B0,
    A,  #: relative amplitude
    theta,
    vA,
    n0,
):
    """
    Generate field with a wave propagating along the x axis at a large angle `theta` with respect to the background magnetic field lying in the x-z plane.

    The initial waveis an Alfven mode in which the magnetic field fluctuation points along the y and z axis and has a relative amplitude $A = \delta B_y / B_0$
    """

    B0k = B0 * cosd(theta)
    B0k1 = B0 * sind(theta)

    Bz_expression = f"{B0k}"
    By_expression = f"{A * B0} * cos({k} * z)"
    Bx_expression = f"{B0k1}"
    pz_expression = 0
    px_expression = 0
    py_expression = f"{A * vA} * cos({k} * z)"
        
    field = picmi.AnalyticInitialField(
        Bx_expression=Bx_expression,
        By_expression=By_expression,
        Bz_expression=Bz_expression,
    )
    
    momentum_expressions = [px_expression, py_expression, pz_expression]
    dist = picmi.AnalyticDistribution(
        density_expression=n0,
        momentum_expressions=momentum_expressions,
    )

    return field, dist

In [ ]:
class AlfvenModes(HybridSimulation):
    test: bool = True
    # Applied field parameters
    dim: int = 2
    B0: float = 100 * 1e-9
    """Initial magnetic field strength (T)"""
    n0: float = 100 * 1e6
    """Initial plasma density (m^-3)"""

    A: float = 0.5  # relative amplitude
    theta: float = 10  # angle with respect to the background magnetic field
    wave_number: int = 3  # wave number

    # Spatial domain
    Lz_norm: float = 128
    Lx_norm: float = 64

    def model_post_init(self, __context):
        """Get input parameters for the specific case desired."""
        if self.test:
            self.m_ion_norm = 100

        super().model_post_init(__context)

        self.k = self.wave_number * 2 * np.pi / self.Lz
        self.setup_run()
        self.dump()

    def setup_field(self):
        """Setup external field"""

        B_ext, _ = init_field_with_plasma(
            k=self.k,
            B0=self.B0,
            A=self.A,
            theta=self.theta,
            vA=self.vA,
            n0=self.n0,
        )
        self._sim.add_applied_field(B_ext)
        return self

    def setup_particle(self):
        """setup the particle"""

        _, dist = init_field_with_plasma(
            k=self.k,
            B0=self.B0,
            A=self.A,
            theta=self.theta,
            vA=self.vA,
            n0=self.n0,
        )

        ions = picmi.Species(
            name="ions",
            charge_state=1,
            mass=self.m_ion,
            initial_distribution=dist,
        )

        self._sim.add_species(
            ions,
            layout=picmi.PseudoRandomLayout(
                grid=self._grid, n_macroparticles_per_cell=self.nppc
            ),
        )
        return self

In [ ]:
simulation = AlfvenModes(
    dim=2,
    theta=0,
)
simulation._sim.write_input_file()

In [ ]:
simulation._sim.step()